<a href="https://colab.research.google.com/github/saishshinde15/NLP/blob/main/HInd_English_translation_Encoder_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv('/content/Hindi_English_Truncated_Corpus.csv')




In [110]:
dataset

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...,...
127602,indic2012,Examples of art deco construction can be found...,आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।
127604,tides,"As for the other derivatives of sulphur , the ...","जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द..."
127605,tides,its complicated functioning is defined thus in...,Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .


In [111]:
dataset=dataset.drop(['source'],axis=1)

In [112]:


dataset= dataset.head(2000)


In [113]:
dataset

,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...
1995,You 'll share in their achievements and be the...,आप उनकी उपलब्धियों में भागीदार होंगे और उनकी ज...
1996,after scientific consensus also the causes of ...,वैज्ञानिक आम सहमति (scientific consensus) होने...
1997,"He's like, “Huh!”",वोह बोला “हैं”
1998,"The neck is heavy , the ears are flabby , the ...","गर्दन इनकी भारी होती है.कान मांसल , सींग मुड़े..."


In [114]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [115]:
tokenizer=Tokenizer(num_words=5000,oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['hindi_sentence'])
y=tokenizer.texts_to_sequences(dataset['hindi_sentence'])

In [116]:
tokenizer=Tokenizer(num_words=5000,oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['english_sentence'])
X=tokenizer.texts_to_sequences(dataset['english_sentence'])

In [117]:
X=pad_sequences(X)

In [118]:
x_dim=X.shape[1]
x_dim

212

In [119]:
y=pad_sequences(y,maxlen=x_dim)

In [120]:
y.shape

(2000, 212)

In [121]:
X.shape

(2000, 212)

In [122]:


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [123]:
len(X_train)

1600

In [124]:
len(y_train)

1600

In [125]:
V=len(tokenizer.word_index)
V

7271

In [128]:
# Define the encoder
encoder_inputs = tf.keras.Input(shape=(x_dim,))
encoder_embedding = tf.keras.layers.Embedding(V+1, 64)(encoder_inputs)
encoder_lstm = tf.keras.layers.LSTM(64, return_sequences=True)(encoder_embedding)
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(64, return_state=True)(encoder_lstm)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = tf.keras.Input(shape=(x_dim,))
decoder_embedding = tf.keras.layers.Embedding(V+1, 64)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(V, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [129]:
# Define the model
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train, y_train], y_train, epochs=10, verbose=1)

# Evaluate the model
model.evaluate([X_test, y_test], y_test, verbose=1)

Epoch 1/10
50/50 [==============================] - 130s 2s/step - loss: 6.4939 - accuracy: 0.8791
Epoch 2/10
50/50 [==============================] - 115s 2s/step - loss: 1.1921 - accuracy: 0.9156
Epoch 3/10
50/50 [==============================] - 119s 2s/step - loss: 0.8517 - accuracy: 0.9156
Epoch 4/10
50/50 [==============================] - 112s 2s/step - loss: 0.8392 - accuracy: 0.9156
Epoch 5/10
50/50 [==============================] - 115s 2s/step - loss: 0.8303 - accuracy: 0.9156
Epoch 6/10
50/50 [==============================] - 113s 2s/step - loss: 0.7984 - accuracy: 0.9156
Epoch 7/10
50/50 [==============================] - 114s 2s/step - loss: 0.7224 - accuracy: 0.9156
Epoch 8/10
50/50 [==============================] - 113s 2s/step - loss: 0.6786 - accuracy: 0.9156
Epoch 9/10
50/50 [==============================] - 112s 2s/step - loss: 0.6433 - accuracy: 0.9156
Epoch 10/10
13/13 [==============================] - 24s 2s/step - loss: 0.6434 - accuracy: 0.9133


[0.6434325575828552, 0.9133018851280212]

In [ ]:
# Generate predictions
predictions = model.predict([X_test, y_test])

# Decode the predictions
decoded_predictions = []
for prediction in predictions:
    decoded_prediction = tokenizer.sequences_to_texts([prediction.argmax(axis=-1)])[0]
    decoded_predictions.append(decoded_prediction)

# Print the predictions
for i in range(len(decoded_predictions)):
    print('English sentence:', X_test[i])
    print('Predicted Hindi sentence:', decoded_predictions[i])



13/13 [==============================] - 14s 1s/step
